# Importing lybraries

In [10]:
# !pip3 install stop-words
# !pip3 install snowballstemmer

import stop_words
import snowballstemmer

stop_words.get_stop_words('pt')
stemmer = snowballstemmer.stemmer('portuguese')
print(stemmer.stemWord('esportistas'))

esport


In [2]:
from textblob import TextBlob
import arff
from io import StringIO
from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier, DecisionTreeClassifier, MaxEntClassifier
import nltk
import pandas as pd

### Importing and setting up hatespeech dataset for training and testing

In [2]:
data = arff.load(open('OffComBR3.arff'))

In [3]:
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

In [4]:
def transform_string_boolen_to_int(x):
    if x == 'yes':
        retorno = 1
    elif x == 'no':
        retorno = 0
    else:
        retorno = x
    return retorno
df['hate'] = df['hate'].apply(transform_string_boolen_to_int)

In [5]:
X = df.drop(['hate'], axis=1)
y = df['sentence']
#X_train, X_test, y_train, y_test = train_test_split(X.values,y,test_size=0.33)
train = df.iloc[:723, :]
test = df.iloc[723:, :]

In [7]:
# this method return the traing data in the TextBlob recommended format
def  build_txblob_array (data):
    txb_array = []
    for d in data.iterrows():
        new_data = (d[1][1], d[1][0])
        txb_array.append(new_data)
    return(txb_array)

In [8]:
# building arrays to train and test the textBlob's NaiveBayesClassifier
train_array = build_txblob_array(train)
test_array = build_txblob_array(test)

### Creating different classifiers and getting its accuracies

In [21]:
# creating classifier
cl_nb = NaiveBayesClassifier(train_array)
cl_dt = DecisionTreeClassifier(train_array)

In [22]:
# testing classifier
cl_nb.accuracy(test_array)

0.6129032258064516

In [23]:
cl_dt.accuracy(test_array)

0.6419354838709678

## Dataset experiments to improve classifications

In [60]:
nltk.download('stopwords')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/marco/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [36]:
train_array[:5]

[('Votaram no PEZAO Agora tomem no CZAO', 1),
 ('cuidado com a poupanca pessoal Lembram o que aconteceu na epoca do Collor ne',
  0),
 ('Sabe o que eu acho engracado os nossos governantes  nao pensam em cortar regalias e beneficios desnecessarios que os favorecem porque sera ne e mais do que claro ate mesmo para quem nao quer enxergar eles sao estao la para defender seus proprios interesses  e os dos empresario o no casso eles tambem ou comecamos a tomar uma atitude para mudar de uma vez por todas essa roubalheira nesse pais o a tendencia e so piorar para o povo porque dinheiro para investimentos nao tem mais para aumentos de salario e regalias nao falta',
  0),
 ('Podiam retirar dos lucros dos bancos ', 0),
 ('CADE O GALVAO PRA NARRAR AGORA   FALIIIIUUUUUUU FALIIIUUUUUUU FALIIUUUUUUU  ',
  0)]

### Lowering the case, removing stopwords and stemming words to get cleaner sentences

In [127]:
def clean_array(array):
    n_array = []
    for cell in array:
        text = ''
        #text = nltk.word_tokenize(w.lower() for w in nltk.word_tokenize(cell[0]) if w not in stopwords)
        for w in nltk.word_tokenize(cell[0]):
            if w not in stopwords:
                text = text + stemmer.stem(w) + ' '
        n_cell = (text.strip(), cell[1])
        n_array.append(n_cell) 
    return n_array

In [129]:
new_train = clean_array(train_array)
new_test = clean_array(test_array)

### Creating new classifiers with the cleanner dataset

In [136]:
new_cl_nb = NaiveBayesClassifier(new_train)
new_cl_dt = DecisionTreeClassifier(new_train)
new_cl_mc = MaxEntClassifier(new_train)

In [134]:
new_cl_nb.accuracy(new_test)

0.6129032258064516

In [133]:
new_cl_dt.accuracy(new_test)

0.6741935483870968

0